In [1]:
#import commands
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [4]:
raw_water_data = pd.read_csv('https://drive.google.com/uc?id=13N4nBi8cZCQUQambCexi0-XArwSghdrj')

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


#### The Relevant Summary of the dataset for exploratory data analysis

In [24]:
# The number of rows that has at least one NaN 
missing_sum = (raw_water_data.isna().sum(axis=1) > 0).sum()
missing_sum

1265

In [26]:
# The total number of rows in the dataset
total_rows = raw_water_data.shape[0]
total_rows

3276

In [27]:
# The percentage of how many rows have missing data in the dataset
percent_missing = (missing_sum / total_rows) * 100
percent_missing

38.614163614163616

In [33]:
# Preprocessor to impute missing values
preprocessor_missing = make_column_transformer(
    (SimpleImputer(), ["ph", "Sulfate", "Trihalomethanes"]),
    remainder='passthrough',
    verbose_feature_names_out=False
)

# Fit and transform the dataset with the preprocessor
preprocessor_missing.fit(raw_water_data)
water_data = preprocessor_missing.transform(raw_water_data)

# Replace the label for each class
water_data["Potability"] = water_data["Potability"].replace({
    0: "Not Potable",
    1: "Potable"
})

# The number of observations in each class
water_data["Potability"].value_counts()

Not Potable    1998
Potable        1278
Name: Potability, dtype: int64

In [35]:
# The dataset that only contains "Not Potable" water
np_water = water_data[water_data["Potability"] == "Not Potable"]

# The dataset that only contains "Potable" water
p_water = water_data[water_data["Potability"] == "Potable"]

# Upsampling to increase the number of observations for "Potable" water 
p_water_upsampled = resample(
    p_water, n_samples=np_water.shape[0]
)

# Concatnating the upsampled dataset and the dataset that only contains "Not Potable" water together
upsampled_water = pd.concat((p_water_upsampled, np_water))

# The number of observations in each class
upsampled_water['Potability'].value_counts()

Potable        1998
Not Potable    1998
Name: Potability, dtype: int64

In [41]:
# The first ten rows of the dataset we are using
water_data = upsampled_water.reset_index(drop=True)
water_data.head(10)

,ph,Sulfate,Trihalomethanes,Hardness,Solids,Chloramines,Conductivity,Organic_carbon,Turbidity,Potability
0,8.544709,350.397082,82.721912,181.413402,31429.379029,7.555030,393.889616,10.247232,2.318152,Potable
1,9.752703,298.082462,51.353052,269.640378,10194.325411,7.978044,580.003534,12.562615,3.422110,Potable
2,6.823642,333.775777,77.227003,170.328172,14053.220679,8.120631,544.011075,13.542213,3.386363,Potable
3,5.678221,418.247228,77.447639,143.186508,18377.008261,7.145335,359.774711,18.714478,5.429782,Potable
4,7.992027,333.775777,71.271231,174.558006,34585.089607,6.587302,543.949141,12.199449,4.647096,Potable
5,7.080795,425.710331,74.885676,158.695209,8721.231885,8.104498,471.073537,15.353950,3.392134,Potable
6,7.833361,393.137872,76.645734,249.340053,13394.731607,8.341670,443.629123,10.983344,4.270973,Potable
7,7.503086,315.974147,43.280476,186.713329,32000.178936,7.641776,473.531611,12.049131,4.305514,Potable
8,7.128062,319.250859,78.468491,191.400821,25227.545153,6.607844,419.304980,12.865366,4.510595,Potable
9,8.238997,350.835986,54.416965,199.594539,25046.798640,9.288020,328.363398,18.743231,3.089924,Potable


In [46]:
# Splitting the dataset into training and testing dataset
water_train, water_test = train_test_split(
    water_data, train_size=0.75, stratify=water_data["Potability"]
)

# The information about the two datasets
print(water_train.info())
print(water_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997 entries, 114 to 2944
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2997 non-null   float64
 1   Sulfate          2997 non-null   float64
 2   Trihalomethanes  2997 non-null   float64
 3   Hardness         2997 non-null   float64
 4   Solids           2997 non-null   float64
 5   Chloramines      2997 non-null   float64
 6   Conductivity     2997 non-null   float64
 7   Organic_carbon   2997 non-null   float64
 8   Turbidity        2997 non-null   float64
 9   Potability       2997 non-null   object 
dtypes: float64(9), object(1)
memory usage: 257.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 2889 to 1151
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               999 non-null    float64
 1   Sulfate          999 non-nul

In [48]:
# The description of all the predictor variables in the training dataset
predictor_vals_summary = water_train.describe()
predictor_vals_summary

,ph,Sulfate,Trihalomethanes,Hardness,Solids,Chloramines,Conductivity,Organic_carbon,Turbidity
count,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000
mean,7.085170,333.982172,66.544205,197.248990,22099.110704,7.139572,424.727178,14.226461,3.958602
std,1.458515,37.468811,15.701748,33.339940,8940.598438,1.575598,81.594795,3.337506,0.791454
min,0.000000,129.000000,0.738000,47.432000,320.942611,0.352000,181.483754,2.200000,1.450000
25%,6.297846,316.541468,56.956939,178.223683,15661.940335,6.156111,363.168538,12.042914,3.413647
50%,7.080795,333.775777,66.396293,197.339559,20920.251561,7.158343,420.357392,14.150363,3.954964
75%,7.833971,351.271267,76.764385,217.394778,27418.781044,8.115768,481.853415,16.437715,4.510578
max,14.000000,481.030642,124.000000,323.124000,61227.196008,13.127000,708.226364,28.300000,6.739000


In [49]:
# The description of the selected predictor variables in the training dataset
selected_predictors_summary = water_train[["Solids", "Conductivity", "Hardness", "Organic_carbon", "Chloramines"]].describe()
selected_predictors_summary

,Solids,Conductivity,Hardness,Organic_carbon,Chloramines
count,2997.000000,2997.000000,2997.000000,2997.000000,2997.000000
mean,22099.110704,424.727178,197.248990,14.226461,7.139572
std,8940.598438,81.594795,33.339940,3.337506,1.575598
min,320.942611,181.483754,47.432000,2.200000,0.352000
25%,15661.940335,363.168538,178.223683,12.042914,6.156111
50%,20920.251561,420.357392,197.339559,14.150363,7.158343
75%,27418.781044,481.853415,217.394778,16.437715,8.115768
max,61227.196008,708.226364,323.124000,28.300000,13.127000


#### The Relevant Visualizations of the dataset for exploratory data analysis

In [54]:
# Unstacked histogram of the dataset
metric_hists = alt.Chart(water_train).mark_bar(opacity = .8).encode(
    alt.X(alt.repeat("repeat"), type='quantitative', bin=True),
    alt.Y("count()", type='quantitative', stack=False),
    color=alt.Color("Potability", scale=alt.Scale(scheme = 'paired'))
).properties(
    width=200,
    height=200
).repeat(
    repeat=['Hardness', 'Solids', 
             'Chloramines', 'Conductivity', 'Organic_carbon'],
    columns=3
).properties(title="Selected Water Quality Metrics by Potability")

metric_hists

alt.RepeatChart(...)